In [1]:
%matplotlib widget

from pathlib import Path

import numpy as np
from wigner import Su2Group

import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import interact, IntSlider, FloatSlider, HBox

In [2]:
dim = 4

In [ ]:
def new_wigner_figure(
    thetas, phis, title: str | None = None, widget: bool = False
) -> go.Figure | go.FigureWidget:
    fig: go.Figure = go.FigureWidget() if widget else go.Figure()

    fig.update_layout(
        title=title,
        title_automargin=True,
        title_font_size=36,
        width=800,
        height=600,
        autosize=False,
        margin=dict(l=0, r=0, b=0, t=0, pad=4),
        scene_camera=dict(eye=dict(x=1, y=1, z=1.2)),
        scene=dict(
            xaxis=dict(showticklabels=False),
            yaxis=dict(showticklabels=False),
            zaxis=dict(showticklabels=False),
        ),
    )

    fig.add_surface(
        x=np.sin(thetas) * np.cos(phis),
        y=np.sin(thetas) * np.sin(phis),
        z=np.cos(thetas),
        cmax=+0.8 * np.sqrt(dim),
        cmin=-0.8 * np.sqrt(dim),
        colorscale=px.colors.diverging.RdBu,
    )

    return fig


density = 100

thetas_1d = np.linspace(0, np.pi, density)
dtheta = np.pi / (density - 1)

phis_1d = np.linspace(0, 2 * np.pi, density)
dphi = 2 * np.pi / (density - 1)

thetas, phis = np.meshgrid(thetas_1d, phis_1d)


fig = new_wigner_figure(thetas, phis, widget=True)


def set_state(rho: np.ndarray[tuple[int, int], np.dtype[np.complexfloating]]):
    global fig, thetas, phis
    fig.data[0].surfacecolor = Su2Group(dim).wigner_transform(thetas, phis, rho)


@interact(
    i=IntSlider(min=0, max=dim - 1, step=1),
    j=IntSlider(min=0, max=dim - 1, step=1),
    theta=FloatSlider(min=0, max=np.pi, step=np.pi / 20, value=np.pi / 2),
    phi=FloatSlider(min=0, max=2 * np.pi, step=2 * np.pi / 20, value=0),
    purity=FloatSlider(min=0, max=1, step=0.05, value=1),
)
def transition(i: int, j: int, theta: float, phi: float, purity: float):
    psi = np.zeros((dim,), dtype=complex)
    if i != j:
        psi[i] = np.cos(theta / 2)
        psi[j] = np.exp(1j * phi) * np.sin(theta / 2)
    else:
        psi[i] = 1

    rho_pure = psi[:, None] * psi[None, :].conj()
    rho_mixed = np.diag(rho_pure.diagonal())
    rho = purity * rho_pure + (1 - purity) * rho_mixed

    set_state(rho)


fig

In [4]:
rho_ghz = np.ones((dim, dim), dtype=float) / dim

assert np.isclose(np.trace(rho_ghz), 1)
assert np.isclose(np.trace(rho_ghz @ rho_ghz), 1)

In [ ]:
density = 500

thetas_1d = np.linspace(0, np.pi, density)
dtheta = np.pi / (density - 1)

phis_1d = np.linspace(0, 2 * np.pi, density)
dphi = 2 * np.pi / (density - 1)

thetas, phis = np.meshgrid(thetas_1d, phis_1d)


figures_dir = Path("figures", f"{dim}")
figures_dir.mkdir(exist_ok=True)

fig = new_wigner_figure(thetas, phis, title=f"GHZ state")
set_state(rho_ghz)
fig.write_image(figures_dir / f"ghz.png", "png", scale=2)

# We only include states a<=b<=dim-a, this is why:
#   1)  We exclude states where a>b,
#       because they are interchangable in |a>+|b>
#   2)  We exclude states where a+b<=dim,
#       becase for states with a+b>dim we can just swap Z axis direction with transformation:
#       n           ->  dim-1-n
#       b+a > dim   ->  dim-b-a _ 2 ->  dim < a+b+2 - and we get back to included states

for a in range((dim + 1) // 2):
    fig = new_wigner_figure(thetas, phis, title=f"|{a}>")
    transition(a, a, np.pi / 2, 0, 1)
    fig.write_image(figures_dir / f"state_{a}.png", scale=2)

    for b in range(a + 1, dim - a):
        fig = new_wigner_figure(thetas, phis, title=f"(|{a}> + |{b}>) / √2")
        transition(a, b, np.pi / 2, 0, 1)
        fig.write_image(figures_dir / f"trans_{a}_{b}.png", scale=2)